In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Datadw=np.array([])
Dataup=np.array([])

num=0

# open the total band file
totbdw = open("band-files/REFORMATTED_BAND_DW.dat")
totbup = open("band-files/REFORMATTED_BAND_UP.dat")
for l in totbdw.readlines():
    Temp = l.split()
    if Temp[0]!='#K-Path':
        if num==0:
            Datadw = np.array(Temp).astype(float)
        else:
            Datadw = np.vstack([Datadw, np.array(Temp).astype(float)])

        num += 1
        
num=0

for l in totbup.readlines():
    Temp = l.split()
    if Temp[0]!='#K-Path':
        if num==0:
            Dataup = np.array(Temp).astype(float)
        else:
            Dataup = np.vstack([Dataup, np.array(Temp).astype(float)])

        num += 1

# close the total band file
totbdw.close()
totbup.close()

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()


# Create and style traces

for i in range(1,Dataup.shape[1]):
    fig.add_trace(go.Scatter( y=Datadw[:,i], name='Banddw'+str(i)))
    fig.add_trace(go.Scatter( y=Dataup[:,i], name='Bandup'+str(i)))


# Edit the layout
fig.update_layout(title='Average High and Low Temperatures in New York',
                   xaxis_title='Month',
                   yaxis_title='Temperature (degrees F)')


fig.show()

In [ ]:
numAtoms=12
numOrbitals=9
numBands=348
numKpoints=400
numStartAtom=5

XDatadw=np.zeros([numKpoints,numBands,numAtoms])
YDatadw=np.zeros([numKpoints,numBands,numAtoms])

AuxSdw=np.zeros([numKpoints,numBands,numOrbitals,numAtoms])

for atom in range(numStartAtom,numStartAtom+numAtoms):
    f = open("PBAND_A"+str(atom)+"_DW.dat")
    for l in f.readlines():
        Temp = l.split()
        if len(Temp)==3:
            bandCount=int(Temp[2])-1
            kCount=0
        if len(Temp)>0:
            if Temp[0]!='#K-Path' and Temp[0]!='#':
                XDatadw[kCount,bandCount,atom-numStartAtom]=float(Temp[0])
                YDatadw[kCount,bandCount,atom-numStartAtom]=float(Temp[1])
                for orb in range(2,11):
                    AuxSdw[kCount,bandCount,orb-2,atom-numStartAtom]
                kCount+=1


XDataup=np.zeros([numKpoints,numBands,numAtoms])
YDataup=np.zeros([numKpoints,numBands,numAtoms])

AuxSup=np.zeros([numKpoints,numBands,numOrbitals,numAtoms])

for atom in range(numStartAtom,numStartAtom+numAtoms):
    f = open("band-files/PBAND_A"+str(atom)+"_UP.dat")
    for l in f.readlines():
        Temp = l.split()
        if len(Temp)==3:
            bandCount=int(Temp[2])-1
            kCount=0
        if len(Temp)>0:
            if Temp[0]!='#K-Path' and Temp[0]!='#':
                XDataup[kCount,bandCount,atom-numStartAtom]=float(Temp[0])
                YDataup[kCount,bandCount,atom-numStartAtom]=float(Temp[1])
                for orb in range(2,11):
                    AuxSup[kCount,bandCount,orb-2,atom-numStartAtom]=float(Temp[orb])
                kCount+=1


In [ ]:
fig = go.Figure()

atom_number=6
# Create and style traces

for i in range(1,XDatadw.shape[1]):
    fig.add_trace(go.Scatter(x=XDatadw[:,i,atom_number], y=YDatadw[:,i,atom_number], name='Banddw'+str(i),marker=dict(size=1150)))

for i in range(1,XDataup.shape[1]):
    fig.add_trace(go.Scatter(x=XDataup[:,i,atom_number], y=YDataup[:,i,atom_number], name='Bandup'+str(i),marker=dict(size=1150)))


# Edit the layout
fig.update_layout(title='Average High and Low Temperatures in New York',
                   xaxis_title='Month',
                   yaxis_title='Temperature (degrees F)')

fig.update_yaxes(range=[-8,3])

fig.show()

In [ ]:
# add Masks to couple the weight with the band energy
Maskdw=np.sum(AuxSdw[:,:,4:9,6],axis=2)
Maskup=np.sum(AuxSup[:,:,4:9,6],axis=2)

In [ ]:
from plotly.offline import plot, iplot, init_notebook_mode

fig = go.Figure()


for i in range(0,336):
    ValuePlotXdw = np.where(XDatadw[:,:,6]*Maskdw < 0.03, np.nan, XDatadw[:,:,6])
    ValuePlotYdw = np.where(YDatadw[:,:,6]*Maskdw < 0.03, np.nan, YDatadw[:,:,6])
    ValuePlotXup = np.where(XDataup[:,:,6]*Maskup < 0.03, np.nan, XDataup[:,:,6])
    ValuePlotYup = np.where(YDataup[:,:,6]*Maskup < 0.03, np.nan, YDataup[:,:,6])
    fig.add_trace(go.Scatter(x=ValuePlotXdw[:,i],
                             y=ValuePlotYdw[:,i],
                             mode='markers',
                             name='Banddw'+str(i),
                             marker=dict(size=30*(Maskdw[:,i]),
                                         line=dict(width=0))))
    fig.add_trace(go.Scatter(x=ValuePlotXup[:,i],
                             y=ValuePlotYup[:,i],
                             mode='markers',
                             name='Bandup'+str(i),
                             marker=dict(size=30*(Maskup[:,i]),
                                         line=dict(width=0))))
    

fig.update_yaxes(range=[-8,3])
#fig.update_xaxes(range=[0,1.5])


iplot(fig)


In [ ]:
ValuePlotY = np.where(YDatadw*Maskdw < 0.02, np.nan, YDatadw)
ValuePlotY = np.where(YDataup*Maskup < 0.02, np.nan, YDataup)

count=0

for i in range(0,ValuePlotY.shape[1]):
    if not np.all(np.isnan(ValuePlotY[:,i])):
        print('Band',i+1)
        count+=1
print('band #: ',count, 'weight = ', YDatadw*Maskdw)
print('band #: ',count, 'weight = ', YDataup*Maskup)